In [1]:
import pyaudio as pa
import wave as wv

In [2]:
import speech_recognition as sp

In [3]:
import pandas as pd
import numpy as np
import spacy
import string
import nltk
import re
from spacy.lang.ru.stop_words import STOP_WORDS
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [4]:
import matplotlib.pyplot as plt
from sklearn import preprocessing
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten
from tensorflow.keras import utils

In [5]:
def Record_Voice():
    global record
    record = pa.PyAudio() # Создаем объект класса PyAudio пакета pyaudio
    
    global stream
    stream = record.open(
        format = FORMAT,
        channels = CHANNELS,
        rate = RATE,
        input = True, # Открываем поток
        frames_per_buffer = CHUNK
    )
    
    print('Идет запись голоса...')
    
    global frames
    frames = []
    
    for i in range (0, int(RATE / CHUNK * RECORD_SECOND)):
        data = stream.read(CHUNK) # Считываем образцы из потока
        frames.append(data)
        
    print('Запись завершена!')

In [6]:
def Completed_Recording():
    stream.stop_stream()
    stream.close()
    record.terminate()

In [7]:
def Save_Record():
    wf = wv.open(OUTPUT_FILENAME, 'wb') # Открываем файл на запись в двоичном режиме
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(record.get_sample_size(FORMAT)) # Возвращает размер в Байтах для указанного значения формата образца
    wf.setframerate(RATE)
    wf.writeframes(b''.join(frames))
    wf.close()

In [8]:
def AudioRecording():
    global CHUNK
    CHUNK = 1024 # Байты
    
    global FORMAT
    FORMAT = pa.paInt16 # 16-и битный формат
    
    global CHANNELS
    CHANNELS = 2 # Индекс микрофона (устройства ввода информации)
    
    global RATE
    RATE = 44100 # Частота записи (кадров)
    
    global RECORD_SECOND
    RECORD_SECOND = 5 # Длительность записи звука в секундах
    
    global OUTPUT_FILENAME
    OUTPUT_FILENAME = 'voice.wav' # Название выходного файла в формате .wav
    
    Record_Voice()
    Completed_Recording()
    Save_Record()

In [9]:
def Text_Conv():
    harvard = sp.AudioFile('voice.wav')
    with harvard as source:
        audio = r.record(source)
        
    global Line
    Line = r.recognize_google(audio, language="ru-RU")

In [10]:
def Text_Write():
    with open("Text.txt", "w") as file:
        file.write(Line)

In [11]:
def ConversionToText():
    global r
    r = sp.Recognizer()
    
    Text_Conv()
    Text_Write()

In [12]:
def List_creating():
    with open("Text.txt", "r") as File:
        for Line in File:
            Texts.append(Line)

In [13]:
def Word_List_creating():
    global Words
    Words = []
    with open("Slov.txt", "r", encoding='UTF-8') as File:
        for Line in File:
            Words.append(Line)
    print(Words)

In [14]:
def Text_Process():
    global Data
    Data = []
    for text in Texts:
        #Блок обработки текста к поиску
        text = text.lower() #Переводим в нижний регистр буквы
        text = re.sub("[«»]", '', text) #Удаление специфичных символов созданного трафика
        text = re.sub(r'\d+', '', text) #Убираем цифры

        translator = str.maketrans('', '', string.punctuation) #Заделка под замену символов
        text = text.translate(translator) #Убираем символы
        text = " ".join(text.split()) #Убираем несколько идущих подряд пробелов
        text = nltk.word_tokenize(text,language = "russian") #Токенизация текста
        text = [token for token in text if token not in STOP_WORDS] #Убираем стоп-слова
        text = " ".join(lemmatizer.lemmatize(token) for token in text) #Лемитизация
        Data.append(text)

In [15]:
def Recording_DataFrame():
    global PrepData
    Train = pd.read_csv('Train.csv')
    PrepData = pd.DataFrame({'Тексты':pd.Series(Data), 'Мошеник':pd.Series(Answer)})
    PrepData = pd.concat([Train, PrepData])

In [16]:
def TF_IDF_Cacl():
    global Data_Texts
    tfIdfTransformer = TfidfTransformer(use_idf=True)
    countVectorizer = CountVectorizer()
    DataVectorized = countVectorizer.fit_transform(PrepData['Тексты'])
    DataVectorized = tfIdfTransformer.fit_transform(DataVectorized)
    Data_Texts = pd.DataFrame(DataVectorized.todense(), columns=countVectorizer.get_feature_names())
    #Data_Texts = pd.DataFrame(DataVectorized.todense(), columns=countVectorizer.get_feature_names_out())
    
    for Word in Data_Texts:
        if any(element in Word + '\n' for element in Words) | any(element in Word for element in Words):
            continue
        else:
            Data_Texts = Data_Texts.drop([Word], axis='columns') 

In [17]:
def SaveTrafics():
    Data_Texts.to_csv('XTrafic.csv')
    PrepData['Мошеник'].to_csv('YTrafic.csv')

In [18]:
def TextProcessing():
    global Texts
    Texts = []
    
    global lemmatizer
    lemmatizer = nltk.stem.WordNetLemmatizer()
    
    global Answer
    Answer = input('Является ли текст мошенническим (Да или Нет): ')
    
    List_creating()
    Word_List_creating()
    Text_Process()
    Recording_DataFrame()
    TF_IDF_Cacl()
    SaveTrafics()

In [19]:
AudioRecording()

Идет запись голоса...
Запись завершена!


In [20]:
ConversionToText()

result2:
{   'alternative': [   {   'confidence': 0.88905942,
                           'transcript': 'так просто говорим что-нибудь сюда и '
                                         'может что-нибудь'},
                       {   'transcript': 'так просто говорил что-нибудь сюда и '
                                         'может что-нибудь'},
                       {   'transcript': 'так просто говорим что-нибудь сюда и '
                                         'может что'}],
    'final': True}


In [21]:
TextProcessing()

Является ли текст мошенническим (Да или Нет): Нет
['банк\n', 'финансы\n', 'технический\n', 'доллары\n', 'проверка\n', 'заявка\n', 'подтверждаете\n', 'паспортные\n', 'доллары\n', 'подозревать\n', 'банкпартнёры\n', 'услуги\n', 'пользовались\n', 'заявка\n', 'продиктовать\n', 'цифровой\n', 'приз\n', 'привяжем\n', 'qrбанк\n', 'процентные\n', 'украли\n', 'помочь\n', 'процентные\n', 'перевод\n', 'правообладателя']
